In [90]:
from gorbslam.common.plotting_utils import create_2d_fig, create_scatter
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
%run setup_path.py
%load_ext autoreload
%autoreload 2

import numpy as np
import plotly.graph_objects as go
from evo.tools import file_interface

from gorbslam.nn.orbslam_processor import ORBSLAMResults, ORBSLAMProcessor
from gorbslam.common.plotting_utils import create_scattermapbox, create_map_fig
from os import path

# TRAJECTORY_NAME = 'vabadussild'
# TRAJECTORY_NAME = 'vabadusepst'
TRAJECTORY_NAME = 'oldtown'

processor = ORBSLAMProcessor(path.join('~/orbslam-out', TRAJECTORY_NAME))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
from sklearn import ensemble
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split


input_scaler = StandardScaler()
X = input_scaler.fit_transform(processor.orbslam.mapping.trajectory)

output_scaler = StandardScaler()
y = output_scaler.fit_transform(processor.orbslam.mapping.trajectory_gt_utm)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=13
)

In [107]:



def search_best_bgr(x_train, y_train):
    estimator = MultiOutputRegressor(ensemble.GradientBoostingRegressor(verbose=1))

    # Define the hyperparameters grid for GradientBoostingRegressor
    param_grid = {
        "estimator__n_estimators": [100, 200, 300],
        "estimator__learning_rate": [0.01, 0.1, 0.2],
        "estimator__max_depth": [3, 4, 5],
        "estimator__min_samples_split": [2, 3, 4],
        "estimator__min_samples_leaf": [1, 2, 3],
    }

    grid_search = GridSearchCV(estimator, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)


    grid_search.fit(X_train, y_train)

    print(f"Best BGR hyperparameters: {grid_search.best_params_}")

    # Retrieve the best model
    return grid_search.best_estimator_


def search_best_rfr(x_train, y_train):
    estimator = MultiOutputRegressor(ensemble.RandomForestRegressor(verbose=1))

    # Define the hyperparameters grid for RandomForestRegressor
    param_grid = {
        "estimator__n_estimators": [10, 50, 100],
        "estimator__max_depth": [None, 10, 20],
        "estimator__min_samples_split": [2, 5, 10],
        "estimator__min_samples_leaf": [1, 2, 4],
    }

    grid_search = GridSearchCV(estimator, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)


    grid_search.fit(X_train, y_train)

    print(f"Best RFR hyperparameters: {grid_search.best_params_}")

    # Retrieve the best model
    return grid_search.best_estimator_


def search_best_svr(x_train, y_train):
    estimator = MultiOutputRegressor(SVR(verbose=1))

    # Define the hyperparameters grid for SVR
    param_grid = {
        "estimator__kernel": ["linear", "poly", "rbf", 'sigmoid'],
        "estimator__degree": [2, 3, 4],
        "estimator__C": [10, 50, 100],
        "estimator__epsilon": [0.06, 0.08, 0.1],
    }

    grid_search = GridSearchCV(estimator, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)


    grid_search.fit(X_train, y_train)

    print(f"Best SVR hyperparameters: {grid_search.best_params_}")

    # Retrieve the best model
    return grid_search.best_estimator_


def predict_real_world_coordinates(estimator, slam_coords_array):
    # Normalize the input SLAM coordinates
    slam_coords_normalized = input_scaler.transform(slam_coords_array)

    # Predict the normalized real-world coordinates
    # real_world_coords_normalized = estimator.predict(slam_coords_normalized)
    real_world_coords_normalized = estimator.predict(slam_coords_normalized)

    # Inverse-transform the normalized real-world coordinates
    real_world_coords = output_scaler.inverse_transform(real_world_coords_normalized)

    return real_world_coords


In [108]:

# estimator = search_best_bgr(X_train, y_train)
# estimator = search_best_rfr(X_train, y_train)
estimator = search_best_svr(X_train, y_train)


Fitting 5 folds for each of 108 candidates, totalling 540 fits
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]...[LibSVM]..[LibSVM]..................................................................................................................
*...
*...
.*..
*.........*
optimization finished, #iter = 20616
obj = -3298.939180, rho = 0.167239
nSV = 913, nBSV = 905
.[LibSVM].........
*..............
*....
*...................
*....................
*.
*
optimization finished, #iter = 13150
.obj = -3330.138266, rho = 0.163781
nSV = 904, nBSV = 898
..[LibSVM].......
*...
*.....*....
*......*
optimization finished, #iter = 25183
obj = -3414.715544, rho = 0.142677
nSV = 913, nBSV = 906
..[LibSVM].
*......*.......
*..............
*..........................................
*.
*........
*.................
*....
*........
*.......................................
*.............
*...*...
*.*
optimization finished, #iter = 19710
obj = -3205.300632, rho = -0.401187
nSV = 832, nBSV = 825
....[LibSV

In [109]:


predicted_0 = predict_real_world_coordinates(estimator, processor.orbslam.localization[0].trajectory)
predicted_1 = predict_real_world_coordinates(estimator, processor.orbslam.localization[1].trajectory)
predicted_2 = predict_real_world_coordinates(estimator, processor.orbslam.localization[2].trajectory)
predicted_3 = predict_real_world_coordinates(estimator, processor.orbslam.localization[3].trajectory)

mode = 'markers'
fig = create_2d_fig(
    traces=[
        create_scatter(processor.orbslam.mapping.trajectory_gt_utm, name='Ground truth', mode=mode),
        create_scatter(predicted_0, name='predicted_0', mode=mode),
        create_scatter(predicted_1, name='predicted_1', mode=mode),
        create_scatter(predicted_2, name='predicted_2', mode=mode),
        create_scatter(predicted_3, name='predicted_3', mode=mode),
    ]
)

fig.show()

In [ ]:
# from matplotlib import pyplot as plt
# from sklearn.metrics import mean_squared_error


# test_score = np.zeros((params["n_estimators"],), dtype=np.float64)
# for i, y_pred in enumerate(reg.staged_predict(X_test)):
#     test_score[i] = mean_squared_error(y_test, y_pred)

# fig = plt.figure(figsize=(6, 6))
# plt.subplot(1, 1, 1)
# plt.title("Deviance")
# plt.plot(
#     np.arange(params["n_estimators"]) + 1,
#     reg.train_score_,
#     "b-",
#     label="Training Set Deviance",
# )
# plt.plot(
#     np.arange(params["n_estimators"]) + 1, test_score, "r-", label="Test Set Deviance"
# )
# plt.legend(loc="upper right")
# plt.xlabel("Boosting Iterations")
# plt.ylabel("Deviance")
# fig.tight_layout()
# plt.show()
